In [ ]:
# 参考 https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-train-amp-aug  训练部分
# 参考 https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-inference-tta  预测部分
# 比赛 https://www.kaggle.com/c/cassava-leaf-disease-classification

# 其他待模仿 https://www.kaggle.com/sachinprabhu/pytorch-resnet50-snapmix-train-pipeline
# 其他待模仿 https://www.kaggle.com/ar2017/pytorch-resnet200d-train-cutmix-fmix

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


import os
import sys
import gc
import math
import pickle
import random
import time
import psutil
import pytz
from datetime import datetime
from collections import defaultdict
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')  # warnings.filterwarnings(action='once')

from tqdm import tqdm, tqdm_notebook

import numpy as np
import pandas as pd
_ = np.seterr(divide='ignore', invalid='ignore')

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns

# 直接在cell中显示图片，支持jpg、png、jpeg等格式，Image('./2.JPG')
from IPython.display import Image  

import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

import tensorflow as tf
from tensorflow import keras

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

def show_process_mem_usage(info_str=''):    ## 显示当前进程占用内存大小
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss
    percent = psutil.virtual_memory().percent
    
    tz = pytz.timezone('Asia/Shanghai')
    now = datetime.now(tz)
    dt_str = now.strftime("%Y-%m-%d %H:%M:%S")
    
    if memory_usage >= 2.**30:
        print(f'{info_str} current process memory usage: {memory_usage/2.**30:.3f} GB, percentage: {percent:.2f}% 【{dt_str}】')
    elif memory_usage >= 2.**20:
        print(f'{info_str} current process memory usage: {memory_usage/2.**20:.3f} MB, percentage: {percent:.2f}% 【{dt_str}】')
    elif memory_usage >= 2.**10:
        print(f'{info_str} current process memory usage: {memory_usage/2.**10:.3f} KB, percentage: {percent:.2f}% 【{dt_str}】')
    else:
        print(f'{info_str} current process memory usage: {memory_usage} B, percentage: {percent:.2f}% 【{dt_str}】')

def logging(*info, file_name='./running_log.txt'):
    log_info = ' '.join([str(s) for s in info])
    with open(file_name, 'a') as f:
        f.write(log_info + '\n')

@contextmanager
def trace(trace_msg):    ## 追踪内存变化和运行时间
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    yield
    m1 = p.memory_info()[0] / 2. ** 30
    delta = m1 - m0
    sign = '+' if delta >= 0 else '-'
    delta = math.fabs(delta)
    trace_msg = str(trace_msg)
    
    tz = pytz.timezone('Asia/Shanghai')
    now = datetime.now(tz)
    dt_str = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{m1:.3f}GB({sign}{delta:.3f}GB):{time.time() - t0:.3f}sec] {trace_msg} 【{dt_str}】", file=sys.stdout)
    
def seed_all(random_seed=42):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True

def keepbusy(num=10000):
    start_t = time.time()
    for i in range(num):
        ftpt(f'i: {i}, taken time: {time.time() - start_t:.7f}')
        time.sleep(60)

def ftpt(msg = 'having run this cell'):  # foot_print   
    tz = pytz.timezone('Asia/Shanghai')
    now = datetime.now(tz)
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f'{dt_string}: {msg}')
    
RANDOM_SEED = 53113
seed_all(RANDOM_SEED)

global_start_t = time.time()

ftpt()

In [ ]:
# def run_one_time(cnt_lst):
#     arr_idx_lst = [i for i in range(len(cnt_lst)) if cnt_lst[i]!=0]
#     random.shuffle(arr_idx_lst)
#     for i in range(int(len(arr_idx_lst)*0.5)):
#         a, b = arr_idx_lst[2*i], arr_idx_lst[2*i+1]
#         if random.randint(0, 1)==0:
#             a, b = b, a
#         if cnt_lst[a]>0 and cnt_lst[b]>0:
#             cnt_lst[a] = cnt_lst[a]-1
#             cnt_lst[b] = cnt_lst[b]+1
#     return cnt_lst
        
# NUM = 100
# cnt_lst = [100]*100

# for i in range(5000000000):
#     zero_num = len([i for i in cnt_lst if i==0])
#     if i%1000==0:
#         print('i: ', i, cnt_lst, 'zero_num: ', zero_num)
#     if zero_num==(NUM-1):
#         print('i: ', i, cnt_lst, 'zero_num: ', zero_num)
#         print('final i: ', i)
#         break
#     cnt_lst = run_one_time(cnt_lst)
    
##############################################################################
    
# cnt_lst

# i:  25675000 [0, 0, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9975, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] zero_num:  98
# final i:  25675301

# i:  48013164 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] zero_num:  99
# final i:  48013164

ftpt()

In [ ]:
package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master/', 
    #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
    '../input/image-fmix/FMix-master'
]
import sys

for pth in package_paths:
    sys.path.append(pth)
    
from fmix import sample_mask, make_low_freq_image, binarise_mask

from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
#from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom

CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1, 
    'device': 'cuda:0' if torch.cuda.is_available() else 'cpu',
}

ftpt()

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.shape
train.head()
train.label.value_counts()

# train = train.sample(n=500)
# train = train.reset_index(drop=True)
# train.shape

ftpt()

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission.head()

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
#     print('im_bgr.shape: ', im_bgr.shape)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

Image('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
# Image('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')

img = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.imshow(img)
plt.show()

In [ ]:
def rand_bbox(size, lam):
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)
    
    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    
    bbx1 = np.clip(cx-cut_w//2, 0, W)
    bby1 = np.clip(cy-cut_h//2, 0, H)
    bbx2 = np.clip(cx+cut_w//2, 0, W)
    bby2 = np.clip(cy+cut_h//2, 0, H)
    return bbx1, bby1, bbx2, bby2


class CassavaDataset(Dataset):
    def __init__(self, df, data_root, 
                 transforms=None, 
                 output_label=True,
                 one_hot_label=False, 
                 do_fmix=False,
                 fmix_params={
                     'alpha': 1.,
                     'decay_power': 3.,
                     'shape': (CFG['img_size'], CFG['img_size']),
                     'max_soft': True,
                     'reformulate': False,
                 }, 
                 do_cutmix=False,
                 cutmix_params={
                     'alpha': 1.,
                 }):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.do_fmix = do_fmix
        self.fmix_params = fmix_params
        self.do_cutmix = do_cutmix
        self.cutmix_params = cutmix_params
        
        self.output_label = output_label
        self.one_hot_label = one_hot_label
        
        if output_label==True:
            self.labels = self.df['label'].values
            if one_hot_label:
                self.labels = np.eye(self.df['label'].max()+1)[self.labels]
                
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        if self.output_label:
            target = self.labels[index]
            
        img = get_img('{}/{}'.format(self.data_root, self.df.loc[index]['image_id']))
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        if self.do_fmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            with torch.no_grad():
                lam = np.clip(np.random.beta(self.fmix_param['alpha'], self.fmix_params['alpha'], 0.6, 0.7))
                
                # Make mask, get mean / std
                mask = make_low_freq_image(self.fmix_params['decay_power'], self.fmix_params['shape'])
                mask = binarise_mask(mask, lam, self.fmix_params['shape'], self.fmix_params['max_soft'])
                
                fmix_ix = np.random.choice(self.df.index, size=1)[0]
                fmix_img = get_img('{}/{}'.format(self.data_root, self.df.iloc[fmix_ix]['image_id']))
                
                if self.transforms:
                    fmix_img = self.transforms(image=fmix_img)['image']
                    
                mask_torch = torch.from_numpy(mask)
                img = mask_torch*img + (1.-mask_torch)*fmix_img
                rate = mask.sum()/CFG['img_size']/CFG['img_size']
                target = rate*target + (1.-rate)*self.labels[fmix_ix]
                
        if self.do_cutmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            with torch.no_grad():
                cmix_ix = np.random.choice(self.df.index, size=1)[0]
                cmix_img = get_img('{}/{}'.format(self.data_root, self.df.iloc[cmix_ix]['image_id']))
                if self.transforms:
                    cmix_img = self.transforms(image=cmix_img)['image']
                    
                lam = np.clip(np.random.beta(self.cutmix_params['alpha'], self.cutmix_params['alpha'], 0.3, 0.4))
                bbx1, bby1, bbx2, bby2 = rand_bbox((CFG['image_size'], CFG['image_size']), lam)
                img[:, bbx1:bbx2, bby1:bby2] = cmix_img[:, bbx1:bbx2, bby1:bby2]
                rate = 1 - (bbx2-bbx2)*(bby2-bby1)/(CFG['img_size']*CFG['img_size'])
                target = rate*target + (1.-rate)*self.labels[cmix_ix]
                
        if self.output_label:
            return img, target
        else:
            return img
                
ftpt()

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
        RandomResizedCrop(CFG['img_size'], CFG['img_size']),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ShiftScaleRotate(p=0.5),
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        CoarseDropout(p=0.5),
        Cutout(p=0.5),
        ToTensorV2(p=1.0),
    ], p=1.0)

def get_valid_transforms():
    return Compose([
        CenterCrop(CFG['img_size'], CFG['img_size'], p=1.0),
        Resize(CFG['img_size'], CFG['img_size']),
        Normalize(mean=[0.485, 0.556, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.0)

ftpt()

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x
    
def prepare_dataloader(df, trn_idx, val_idx, data_root='../input/cassava-leaf-disease-classification/train_images/'):
    from catalyst.data.sampler import BalanceClassSampler
    train_ = df.loc[trn_idx, :].reset_index(drop=True)
    valid_ = df.loc[val_idx, :].reset_index(drop=True)
    train_ds = CassavaDataset(train_, data_root, transforms=get_train_transforms(),
                              output_label=True, one_hot_label=False, do_fmix=False, 
                              do_cutmix=False)
    valid_ds = CassavaDataset(valid_, data_root, transforms=get_valid_transforms(),
                              output_label=True)
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,
        num_workers=CFG['num_workers'],
#         sampler=BalanceClassSampler(labels=train_['label'].values, mode='downsampling')
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds,
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, 
                    scheduler=None, schd_batch_update=False):
    model.train()
    
    t = time.time()
    running_loss = None
    
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        with autocast():
            image_preds = model(imgs)
            loss = loss_fn(image_preds, image_labels)
            scaler.scale(loss).backward()
            
            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01
                
            if ((step+1)%CFG['accum_iter']==0) or ((step+1)==len(train_loader)):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
                if scheduler is not None and schd_batch_update:
                    description = f'epoch {epoch} loss: {running_loss:.4f}'
                    pbar.set_description(description)
    if scheduler is not None and not schd_batch_update:
        scheduler.step()
        
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    print('in valid_one_epoch()')
    model.eval()
    
    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_prob_all = []
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)
        image_preds_prob_all += [image_preds.detach().cpu().numpy()]
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        loss = loss_fn(image_preds, image_labels)
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]
        
        if ((step+1)%CFG['verbose_step']==0) or ((step+1)==len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
            
    image_preds_prob_all = np.concatenate(image_preds_prob_all)
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print('validation multi-class log_loss = {:.4f}'.format(log_loss(image_targets_all, image_preds_prob_all)))
    print('validation multi-class accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()
           
        
class MyCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean'):
        super().__init__(weight=weight, reduction=reduction)
        self.weight = weight
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        lsm = F.log_softmax(inputs, -1)
        
        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)
            
        loss = -(targets * lsm).sum(-1)
        if self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
            
        return loss
    
ftpt()

In [ ]:
if __name__=='__main__':
    seed_everything(CFG['seed'])
    folds = StratifiedKFold(n_splits=CFG['fold_num'], shuffle=True, random_state=CFG['seed']).split(
        np.arange(train.shape[0]), train.label.values)
    
    for fold, (trn_idx, val_idx) in enumerate(folds):
        fold_start_t = time.time()
        print('in loop, fold: ', fold)
        # we'll train fold 0 and fold 1 first
        if fold > 1:
            break
            
        print(f'Training with {fold} started')
        print(f'len(trn_idx): {len(trn_idx)} len(val_idx): {len(val_idx)}')
        train_loader, val_loader = prepare_dataloader(train, trn_idx, val_idx, 
                                                      data_root='../input/cassava-leaf-disease-classification/train_images/')
        device = torch.device(CFG['device'])
        model = CassvaImgClassifier(CFG['model_arch'], train.label.nunique(), pretrained=True).to(device)
        scaler = GradScaler()
        optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['T_0'], T_mult=1, eta_min=CFG['min_lr'], last_epoch=-1)
        
        loss_tr = nn.CrossEntropyLoss().to(device)
        loss_fn = nn.CrossEntropyLoss().to(device)
        
        for epoch in range(CFG['epochs']):
            train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=False)
            
            with torch.no_grad():
                valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)
                
            torch.save(model.state_dict(), '{}_fold_{}_{}'.format(CFG['model_arch'], fold, epoch))
            
        del model, optimizer, train_loader, val_loader, scaler, scheduler
        torch.cuda.empty_cache()
        fold_start_t = time.time()
        print(f'fold: {fold}, cost time: {time.time()-fold_start_t:.5f}')
        
test = pd.DataFrame()
test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
test['label'] = 1
print('test.shape: ', test.shape)
test.to_csv('submission.csv', index=False)

print('this run total cost time: ', time.time() - global_start_t)

ftpt()